In [16]:
pip install tiktoken chromadb

  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/b8/eb/234646d9eefda8a500d0fd88b05bf625a90ed18054124349db26e558276e/tiktoken-0.5.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for regex>=2022.1.18 from https://files.pythonhosted.org/packages/b8/ad/3398312096118c4e62a5827664e52a04d5068e84d04142dd4a0da8a567ae/regex-2023.10.3-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ------------------ ------------------- 20.5/42.0 kB 320.0 kB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 503.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/759.8 kB ? eta -:--:--
   --- ------------------------------------ 61.4/759.8 kB 1.7 MB/s eta 0:00:01
   ----------------- ---------------------- 327.7/759.8 kB 4.1 MB/s eta 0:00:01
   ------------------------------------ --- 686.1/759.8 kB 5.4 MB/s eta 0:00:01
   --------------------------

In [10]:
# you need openai key

In [18]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser

In [12]:
from langchain.document_loaders import TextLoader

loader = TextLoader("categories_tagging.md")

In [13]:
# Split documents

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(loader.load())

In [17]:
# Embed and store splits

from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [20]:
# LLM

prompt_template = """Use the context below to assign a category that is relevant.
After assigning a category, I want you to assign three tags in the chosen category that are relevant to the context

Context: {context}
info: {tagging}
"""


PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "tagging"])

runnable_chain = PROMPT | ChatOpenAI() | StrOutputParser()

In [21]:
def answer_question(context, index, chain):
    """
    Takes in query, index to search from, and llm chain to generate answer
    """
    answer = chain.invoke({"context": context, "tagging": index})
    return answer

In [26]:
info = """
SECTION 1. Chapter 20 Section 23 of the General Laws, as appearing in the 2020 Official Edition, is hereby amended by inserting the following after section (f): -
	(g) Notwithstanding any general or special law to the contrary, the department of agricultural resources, 
    with the approval of the co-holder, if any, in its sole discretion, may allow for storm water mitigation or nitrogen mitigation purposes, 
    in nitrogen sensitive embayment watersheds, uncultivated, planted or other storm water mitigation infrastructure within 200 feet of any resource area in any parcel 
    that has been accepted into the Massachusetts Agricultural Preservation Restriction program."""

answer = answer_question(info, vectorstore, runnable_chain)
print(info)
print(answer)


SECTION 1. Chapter 20 Section 23 of the General Laws, as appearing in the 2020 Official Edition, is hereby amended by inserting the following after section (f): -
	(g) Notwithstanding any general or special law to the contrary, the department of agricultural resources, 
    with the approval of the co-holder, if any, in its sole discretion, may allow for storm water mitigation or nitrogen mitigation purposes, 
    in nitrogen sensitive embayment watersheds, uncultivated, planted or other storm water mitigation infrastructure within 200 feet of any resource area in any parcel 
    that has been accepted into the Massachusetts Agricultural Preservation Restriction program.
Category: Environmental Regulations

Tags:
1. Storm water mitigation
2. Nitrogen mitigation
3. Massachusetts Agricultural Preservation Restriction program
